In [ ]:
import os
import urllib.request

In [ ]:
name = '{{ NAME }}'
hsla_tarball_url = "https://archive.stsci.edu/missions/hst/spectral_legacy/datapile_05-15-2018_COS/" + name + "/" + name + "_target.tar.gz"
hsla_tarball = name + "_target.tar"
gzipball, headers = urllib.request.urlretrieve(hsla_tarball_url, filename=hsla_gzipped)
!gunzip $hsla_tarball
!tar -xvf $hsla_tarball

## Now you should have some images, tables, and fits files in your pwd
